In [2]:
!pip install torch
!pip install git+https://github.com/openai/shap-e.git
!pip install pytorch3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [4]:
from shap_e.util.notebooks import decode_latent_mesh

In [ ]:
print(torch.version.cuda)

12.4


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:12<00:00, 72.0MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [7]:
batch_size = 4
guidance_scale = 15.0
prompt = "girrafe"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [8]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAMy/fsu4bMa2br2uZcGsY7ulWbWkZqKbaaiYVKaUUZ6UVKOTTq…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAMW4mr6xkruvjrmrhbKnjaqijKmfhqicgKOYfp6Sd6iRaKORY6…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAL22h7qzjbmxl7Kqj7Oqf7Kmg6ulcqmkg6mgeKaaa6Gaa6CZcK…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAMSxe7unbaqkerOgZqafe6SedaSaZ6uUXJ6WcZuYa5qUZ6mPVa…

In [9]:
size = 64 # this is the size of the renders; higher values take longer to render.
for i, latent in enumerate(latents):
  image = decode_latent_mesh(xm, latent)
  display(image)

/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


TorchMesh(verts=tensor([[-0.9632, -0.0078,  0.6279],
        [-0.9662, -0.0078,  0.6434],
        [-0.9630, -0.0078,  0.6589],
        ...,
        [ 0.9690,  0.0078, -0.4607],
        [ 0.9690,  0.0233, -0.5268],
        [ 0.9690,  0.0233, -0.4763]], device='cuda:0'), faces=tensor([[11530,  3800,     0],
        [ 3800,  3801,     0],
        [ 3801,     1,     0],
        ...,
        [11528, 11529,  3798],
        [11529,  3799,  3798],
        [ 3799, 11529, 15581]], device='cuda:0'), vertex_channels={'R': tensor([0.0691, 0.0808, 0.1058,  ..., 0.1997, 0.6364, 0.2553], device='cuda:0'), 'G': tensor([0.0549, 0.0635, 0.0854,  ..., 0.1543, 0.5832, 0.2071], device='cuda:0'), 'B': tensor([0.0247, 0.0287, 0.0412,  ..., 0.0530, 0.2714, 0.0755], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.9698, -0.0388, -0.7209],
        [-0.9702, -0.0388, -0.7054],
        [-0.9704, -0.0388, -0.6899],
        ...,
        [ 0.9380,  0.0078,  0.7109],
        [ 0.9380,  0.0233,  0.6895],
        [ 0.9380,  0.0233,  0.6915]], device='cuda:0'), faces=tensor([[12318,  4294,     0],
        [ 4294,  4295,     0],
        [ 4295,     1,     0],
        ...,
        [ 4292, 12316, 16471],
        [16472, 12317,  4293],
        [ 4293, 12317, 16473]], device='cuda:0'), vertex_channels={'R': tensor([0.0081, 0.0081, 0.0093,  ..., 0.0281, 0.0284, 0.0281], device='cuda:0'), 'G': tensor([0.0045, 0.0044, 0.0050,  ..., 0.0169, 0.0193, 0.0186], device='cuda:0'), 'B': tensor([0.0026, 0.0025, 0.0027,  ..., 0.0083, 0.0110, 0.0104], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.9130, -0.0078,  0.2868],
        [-0.9071, -0.0078,  0.3023],
        [-0.9079,  0.0078,  0.2713],
        ...,
        [ 0.9380, -0.0078, -0.5202],
        [ 0.9380,  0.0078, -0.5887],
        [ 0.9380,  0.0078, -0.5214]], device='cuda:0'), faces=tensor([[12274,  4280,     0],
        [ 4280,  4281,     0],
        [ 4281,     1,     0],
        ...,
        [12272, 12273,  4278],
        [12273,  4279,  4278],
        [ 4279, 12273, 16645]], device='cuda:0'), vertex_channels={'R': tensor([0.1797, 0.1830, 0.1903,  ..., 0.4299, 0.1914, 0.3368], device='cuda:0'), 'G': tensor([0.1371, 0.1387, 0.1436,  ..., 0.3834, 0.1742, 0.3064], device='cuda:0'), 'B': tensor([0.0512, 0.0496, 0.0534,  ..., 0.2383, 0.0916, 0.1810], device='cuda:0')}, face_channels={})

TorchMesh(verts=tensor([[-0.9717, -0.0078, -0.5194],
        [-0.9740,  0.0078, -0.5194],
        [-0.9725,  0.0078, -0.5039],
        ...,
        [ 0.9535, -0.0078,  0.7092],
        [ 0.9535,  0.0078,  0.6939],
        [ 0.9535,  0.0078,  0.7107]], device='cuda:0'), faces=tensor([[11952,  3688,     0],
        [ 3688, 11953,     0],
        [    0,     1, 11952],
        ...,
        [ 3687, 15747, 15745],
        [15746, 11951,  3687],
        [ 3687, 11951, 15747]], device='cuda:0'), vertex_channels={'R': tensor([0.2317, 0.2444, 0.1808,  ..., 0.3843, 0.3609, 0.3710], device='cuda:0'), 'G': tensor([0.1873, 0.2033, 0.1475,  ..., 0.2951, 0.2847, 0.2826], device='cuda:0'), 'B': tensor([0.0693, 0.0785, 0.0537,  ..., 0.1015, 0.1029, 0.0953], device='cuda:0')}, face_channels={})

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/usr/local/lib/python3.11/dist-packages/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(
